In [33]:
import collections


ENV_NAME = "FrozenLake-v0"

GAMMA = 0.9
TEST_EPISODES = 20

In [34]:
import gym 

env = gym.make(ENV_NAME)
rewards = collections.defaultdict(float)
transits = collections.defaultdict(collections.Counter)
values = collections.defaultdict(float)

In [35]:
def play_random_episode(count):
  state = env.reset()
  for _ in range(count):
    # env.render()
    # n = env.action_space.n
    action = env.action_space.sample()
    new_state, reward, is_done, _ = env.step(action)

    # print(state)
    # print(action)
    #print(new_state)

    # collect the reward table
    rewards[(state, action, new_state)] = reward
    transits[(state, action)][new_state] += 1

    # set current state to be new_state if the episode is not done
    state = env.reset() if is_done else new_state
  
  # print(rewards)
  # print(len(rewards))
  ##print(transits)

# Test function
play_random_episode(3)

In [36]:
# calculate the state value for a given action and transition probability using the bellman equation
def calculate_state_value(state, action):
  # get the transtions using the transition dict
  transitions = transits[(state, action)]
  total = sum(transitions.values())
  state_value = 0.0
  # Iterate over the possible transitions and update the state values
  for new_state, count in transitions.items():
    state_value = state_value + rewards[(state, action, new_state)] + GAMMA * (count / total) * values[new_state]

  return state_value

In [37]:
# value iteration; For every state "s" find the state value from the bellman optimal equation
def value_iteration():
  # Iterate for all states
  for state in range(env.observation_space.n):
    # get state values for all possible actions for a given state
    state_values = [calculate_state_value(state, action) for action in range(env.action_space.n)]
    # find the action that maximises the state value and update the value table
    values[state] = max(state_values)

In [38]:
# Find the best action given a state by calculating the state values for each actions
def select_best_action(state):
  best_action, best_value = None, None
  for action in range(env.action_space.n):
    state_value = calculate_state_value(state, action)
    if best_value == None or best_value < state_value:
      best_value = state_value
      best_action = action

  return best_action

In [39]:
# Play a set of episodes using the value table
def play_episode(env):
  state = env.reset()
  total_reward = 0.0
  while True:
    # select the best action for a given state from the value table
    action = select_best_action(state)
    # play the action
    new_state, reward, is_done, _ = env.step(action)
    # Update the total reward
    total_reward += reward
    # set next state as current state
    state = new_state
    # rewards[(state, action, new_state)] = reward
    # transits[(state, action)][new_state] += 1
    # if episode done; break
    if is_done:
      break

  return total_reward

In [40]:
## Algorithm
## 1. Play random episodes and explore the env
## 2. Update the value table for the values that maximises the state value
## 3. Exploit the env using the best actions and get rewarded
## 4. Check if we have reached the reward bound and if so terminate
test_env = gym.make(ENV_NAME)
# writer = SummaryWriter(comment="-v-iteration")

iter_no = 0
best_reward = 0.0
while True:
    #test_env.render()
    iter_no += 1
    ## Explore the env by playing random actions
    ## and update the transit and reward tables
    play_random_episode(100)
    ## Find the new values of the states after random actions
    value_iteration()
    #print(values)
    #print(transits)
    #print(rewards)
    reward = 0.0
    for _ in range(TEST_EPISODES):
        ## Play an episode, this is the exploitation stage
        ## in this stage, agent utilises the best actions that maximises the state value
        reward += play_episode(test_env)
        #print(reward)
    reward /= TEST_EPISODES
    # writer.add_scalar("reward", reward, iter_no)
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
        best_reward = reward
    if reward > 0.80:
        print("Solved in %d iterations!" % iter_no)
        break

Best reward updated 0.000 -> 0.150
Best reward updated 0.150 -> 0.200
Best reward updated 0.200 -> 0.500
Best reward updated 0.500 -> 0.650
Best reward updated 0.650 -> 0.700
Best reward updated 0.700 -> 0.750
Best reward updated 0.750 -> 0.900
Solved in 179 iterations!
